<a href="https://colab.research.google.com/github/virendrasinh734/medicinal_plant_identification/blob/main/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install wget
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/748f8jkphb-3.zip

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!unzip 748f8jkphb-3.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import tensorflow as tf
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset",
    shuffle=True,
    batch_size=32,
    image_size=(299, 299),
)

labels = dataset.class_names
labels

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import numpy as np

for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())
    break

(32, 299, 299, 3)
[49 49  3 49 56 63 23  9 70 69 49 79 28  5 67  7 26 23 58 21 29 57 42 76
  4 36 50  1 61 49 35  4]


In [ ]:
train_size = int(0.8 * len(dataset))
test_size = int(0.2 * len(dataset))
train_size, test_size

(172, 43)

In [ ]:
def get_dataset_partisions_tf(ds, train_split=0.8, test_split=0.2, shuffle=True, shuffle_size=10000):
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * len(ds))
    test_size = int(test_split * len(ds))
    train_ds = ds.take(train_size)
    test_ds = ds.skip(train_size)
    val_ds = test_ds.skip(test_size)
    test_ds = test_ds.take(test_size)
    return train_ds, test_ds, val_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partisions_tf(dataset, train_split=0.7, test_split=0.15, shuffle=True, shuffle_size=10000)
len(train_ds), len(test_ds), len(val_ds)

(151, 33, 32)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(299, 299),
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])

In [ ]:
base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    input_shape=(299, 299, 3),
    include_top=False,
    pooling='avg',
    classifier_activation='softmax',
    classes=len(labels)
)

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

inputs = tf.keras.Input(shape=(299, 299, 3))
x = resize_and_rescale(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(len(labels), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=32,
    epochs=20
)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 80)                10320     
                                                                 
Total params: 22075376 (84.21 MB)
Trainable params: 272592 (1

In [ ]:
history

In [ ]:
model.save("my_mmodel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.evaluate(test_ds)

33/33 [==============================] - 204s 130ms/step - loss: 0.1578 - accuracy: 0.9792


[0.15783238410949707, 0.9791666865348816]

In [ ]:
model.save("my_h5_model.h5")